In [56]:
import os
import cv2
from random import shuffle
from tqdm import tqdm
import numpy as np

In [57]:
cv2_base_dir = os.path.dirname(os.path.abspath(cv2.__file__))
haar_model = os.path.join(cv2_base_dir, 'data/haarcascade_frontalface_default.xml')
face_classifier = cv2.CascadeClassifier(haar_model)
cap = cv2.VideoCapture(0)
id = 1
name = "Bharat"
img_id = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture frame from webcam.")
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cropped_face = frame[y:y+h, x:x+w]
    img_id += 1
    face = cv2.resize(cropped_face, (150, 150))
    gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
    file_name_path = f"DataSet/{name}" + "." + str(id) + "."+ str(img_id) + ".jpg"
    cv2.imwrite(file_name_path, face)
    cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Cropped Face", face)
    if cv2.waitKey(1) == ord('q') or int(img_id) == 100:
        break
cap.release()
cv2.destroyAllWindows()
print("Done")
 


Done


In [58]:
def my_label(image_name):
    name = image_name.split('.')[-3]
    if name == "Bharat":
        return np.array([1,0,0])
    elif name == "Shyam":
        return np.array([0,1,0])
    elif name == "Khedup":
        return np.array([0,0,1])
        

In [59]:
def my_data():
    data= []
    for img in tqdm(os.listdir("DataSet")):
        path = os.path.join("DataSet", img)
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50, 50))
        data.append([np.array(img_data), my_label(img)])
    shuffle(data)
    return data

In [60]:
data = my_data()
train = data[:240]
test = data[240:]
X_train = np.array([i[0] for i in train]).reshape(-1, 50, 50, 1)
X_test = np.array([i[1] for i in train])
y_train = np.array([i[0] for i in train]).reshape(-1, 50, 50, 1)
y_test = np.array([i[1] for i in train])

print(X_train)
print(Y_test)

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 193.93it/s]

[[[[70]
   [72]
   [70]
   ...
   [87]
   [87]
   [88]]

  [[69]
   [69]
   [69]
   ...
   [88]
   [88]
   [89]]

  [[69]
   [69]
   [70]
   ...
   [90]
   [88]
   [92]]

  ...

  [[49]
   [49]
   [50]
   ...
   [69]
   [70]
   [69]]

  [[47]
   [48]
   [49]
   ...
   [70]
   [70]
   [68]]

  [[49]
   [50]
   [48]
   ...
   [71]
   [69]
   [71]]]


 [[[67]
   [68]
   [67]
   ...
   [83]
   [82]
   [84]]

  [[69]
   [66]
   [69]
   ...
   [85]
   [86]
   [87]]

  [[70]
   [70]
   [67]
   ...
   [85]
   [85]
   [88]]

  ...

  [[44]
   [43]
   [43]
   ...
   [18]
   [63]
   [67]]

  [[44]
   [44]
   [44]
   ...
   [22]
   [66]
   [66]]

  [[44]
   [44]
   [44]
   ...
   [31]
   [62]
   [68]]]


 [[[67]
   [68]
   [67]
   ...
   [83]
   [82]
   [84]]

  [[69]
   [66]
   [69]
   ...
   [85]
   [86]
   [87]]

  [[70]
   [70]
   [67]
   ...
   [85]
   [85]
   [88]]

  ...

  [[44]
   [43]
   [43]
   ...
   [18]
   [63]
   [67]]

  [[44]
   [44]
   [44]
   ...
   [22]
   [66]
   [66]]

  [[44

In [61]:
# import warnings
# warnings.filterwarnings("ignore")
from PIL import Image as pil
from pkg_resources import parse_version

if parse_version(pil.__version__)>=parse_version('10.0.0'):
    pil.ANTIALIAS=pil.LANCZOS

import tensorflow as tf
from tensorflow.keras import layers
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

In [62]:
# tf.compat.v1.reset_default_graph()
# convnet = layers.Input(shape=(50, 50, 1))
# convnet = layers.Conv2D(32, 5, activation= 'relu')(convnet)
# convnet = layers.MaxPooling2D(5)(convnet)
# convnet = layers.Conv2D(64, 5, activation= 'relu')(convnet)
# convnet = layers.MaxPooling2D(5)(convnet)
# convnet = layers.Conv2D(128, 5, activation= 'relu')(convnet)
# convnet = layers.MaxPooling2D(5)(convnet)
# convnet = layers.Conv2D(64, 5, activation= 'relu')(convnet)
# convnet = layers.MaxPooling2D(5)(convnet)
# convnet = layers.Conv2D(32, 5, activation= 'relu')(convnet)
# convnet = layers.MaxPooling2D(5)(convnet)

# convnet = tf.contrib.layers.fully_connected(convnet, 1024, activation_fn = 'relu')
# convnet = layers.Dropout(0.8)(convnet)
# convnet = tf.contrib.layers.fullyconnected(convnet, 3, activation_fn = 'softmax')
# convnet = tf.keras.models.Sequencial(optimizer='adam',learning_rate=0.001, loss="categorical_crossentropy")

In [65]:
# tf.reset_default_graph()
tf.compat.v1.reset_default_graph()
convnet = input_data(shape=[50, 50, 1])
convnet = conv_2d(convnet, 32, 5, activation= 'relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)
convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)
convnet = fully_connected(convnet, 3, activation='softmax')
convnet = regression(convnet, optimizer='adam',learning_rate=0.001, loss="categorical_crossentropy")
model = tflearn.DNN(convnet, tensorboard_verbose=1)
model.fit(X_train, y_train, n_epoch=12, validation_set=(X_test, y_test),show_metric = True, run_id= "FRS")

---------------------------------
Run id: FRS
Log directory: /tmp/tflearn_logs/
INFO:tensorflow:Summary name Accuracy/ (raw) is illegal; using Accuracy/__raw_ instead.
---------------------------------
Training samples: 100
Validation samples: 100
--


ValueError: Cannot feed value of shape (64, 50, 50, 1) for Tensor TargetsData/Y:0, which has shape (?, 3)

In [ ]:
def data_for_visualization():
    Vdata = []
    for img in tqdm(os.listdir("visualization")):
        path = os.path.join("visualization", img)
        img_num = img.split('.')[0] 
        img_data = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        img_data = cv2.resize(img_data, (50,50))
        Vdata.append([np.array(img_data), img_num])
    shuffle(Vdata)
    return Vdata

In [ ]:
import matplotlib.pyplot as plt   # installation command: pip install matplotlib
 
fig = plt.figure(figsize=(20,20))
for num, data in enumerate(Vdata[:20]):
    img_data = data[0]
    y = fig.add_subplot(5,5, num+1)
    image = img_data
    data = img_data.reshape(50,50,1)
    model_out = model.predict([data])[0]
     
    if np.argmax(model_out) == 0:
        my_label = 'Bharat'
    elif np.argmax(model_out) == 1:
        my_label = 'Shyam'
    else:
        my_label = 'Khedup'
         
    y.imshow(image, cmap='gray')
    plt.title(my_label)
     
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture frame from webcam.")
        break
    data = frame.reshape(

    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break
    
    # name = input("Enter your name: ")
    # workbook = xlsxwriter.Workbook(f"{name}.xlsx")
    # worksheet = workbook.add_worksheet()
    
    # if flag:
    #     worksheet.write('Bharat', 'Present')
    # else:
    #     worksheet.write('Bharat', "Absent")
    # workbook.close()
    # print("Done")
cap.release()
cv2.destroyAllWindows()